PY DATABASE CONFIGURATION

In [ ]:
#DATABASE CONFIGURATION
#VERİTABANI OLUŞTURMA VE VERİ AKTARMA
import os
import pandas as pd
import sqlite3
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'example databases\worldcities.csv')
df = pd.read_csv(#dataframe'i oku
    file_path
)
#sayılsal kolonları dönüştür 
df["population"] =pd.to_numeric(df["population"], errors='coerce')
#sqlite veritabanına bağlan
connection = sqlite3.connect("database_query.db")
#dataframii tabloya aktar
df.to_sql("cities", connection, if_exists="replace", index=False)

#SORGULARI YAZABİLECEĞİMİZ BİR SORGU FONKSİYONU YAZALIM
def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None
#cursor(imleç)
cursor = connection.cursor()

-SQL SORGU OPERATÖRLERİ-

In [ ]:
"""                       
1. SELECT
   - Tablodaki kolonları seçmek için kullanılır.

2. WHERE
   - Kayıtları belirli bir koşula göre filtrelemek için kullanılır.

3. ORDER BY
   - Sonuçları artan veya azalan sırada düzenlemek için kullanılır.

4. GROUP BY
   - Kayıtları belirli bir kolona göre gruplamak için kullanılır.

5. HAVING
   - GROUP BY sonrasında gruplar üzerinde koşul uygulamak için kullanılır.

6. LIKE
   - Metin karşılaştırmalarında benzer ifadeleri filtrelemek için kullanılır.

7. LIMIT
   - Sonuç kümesinden belirli sayıda kayıt döndürmek için kullanılır.

8. JOIN
   - İlişkili tabloları belirli bir koşula göre birleştirmek için kullanılır.

9. ALT SORGULAR (SUBQUERIES)
   - Bir sorgu içinde yer alan başka bir sorgudur; filtreleme, karşılaştırma veya hesaplama amaçlı kullanılır.
"""

In [ ]:
#örnek tablo CREATE
try:
    cursor.execute("""
                CREATE TABLE Ogrenciler (
                    ogrenci_id INT PRIMARY KEY,
                    ogrenci_no VARCHAR(20) UNIQUE NOT NULL,
                    ad VARCHAR(50),
                    soyad VARCHAR(50),
                    bolum VARCHAR(100)
                );
""")
    cursor.execute("""
                CREATE TABLE Dersler (
                    ders_id INT PRIMARY KEY,
                    ders_kodu VARCHAR(10) UNIQUE,
                    ders_adi VARCHAR(100),
                    kredi INT
                );
""")
    cursor.execute("""
                CREATE TABLE AkademikKayitlar (
                    kayit_id INT PRIMARY KEY,
                    ogrenci_id INT,
                    ders_id INT,
                    donem VARCHAR(10),
                    notu INT,
                    FOREIGN KEY (ogrenci_id) REFERENCES Ogrenciler(ogrenci_id),
                    FOREIGN KEY (ders_id) REFERENCES Dersler(ders_id)
                );
""")
except Exception as e:
    print(e)
connection.commit()


In [ ]:
#tablolara veri ekledik
try:
    cursor.execute("""
                INSERT INTO Ogrenciler (ogrenci_id, ogrenci_no, ad, soyad, bolum)
                VALUES
                (1, '2023001', 'Ali',   'Yılmaz', 'Bilgisayar Mühendisliği'),
                (2, '2023002', 'Ayşe',  'Demir',  'Yazılım Mühendisliği'),
                (3, '2023003', 'Mehmet','Kaya',   'Bilgisayar Mühendisliği'),
                (4, '2023004', 'Elif',  'Çetin',  'Elektrik-Elektronik Mühendisliği');
""")
    cursor.execute("""
                            
                INSERT INTO Dersler (ders_id, ders_kodu, ders_adi, kredi)
                VALUES
                (1, 'BLM101', 'Veri Yapıları', 4),
                (2, 'BLM102', 'Veritabanı Sistemleri', 3),
                (3, 'BLM201', 'Algoritmalar', 4),
                (4, 'MAT101', 'Lineer Cebir', 3);
""")
    cursor.execute("""
                INSERT INTO AkademikKayitlar (kayit_id, ogrenci_id, ders_id, donem, notu)
                VALUES
                (1, 1, 1, '2024G', 85),
                (2, 1, 2, '2024G', 90),
                (3, 1, 3, '2024G', 78),
                (4, 2, 1, '2024G', 70),
                (5, 2, 2, '2024G', 88),
                (6, 3, 1, '2024G', 60),
                (7, 3, 4, '2024G', 75),
            (8, 4, 4, '2024G', 82);
""")
except Exception as e:
    print(e)

In [ ]:
run_query("select count(*) as count from ogrenciler")
run_query("select count(*) as count from Dersler")
run_query("select count(*) as count from AkademikKayitlar")

In [ ]:
#   -RELATIONAL ALGEBRA-
""""
Seçim(σ) 
Projeksiyon(π)
Birleşim(U)
Küme Farkı(-)
Küme Kesişimi(∩)
Yeniden Adlandırma(ρ)
Kartezien Çarpım(×)
Join(⋈)
    Condition Join(⋈c)
    Equijoin(⋈e)
    Natural Join(⋈)
    Outer Join(-⋈-)
        Left Outer Join(⟕)
        Right Outer Join(⟖)
        Full Outer Join(⟗)
    Semi Join(⋉)
    Anti Join(▷)
"""

ÖRNEK SORGULAR

In [ ]:
#popülasyonu 1 milyondan fazla olan şehirleri al ilk 5 tanesini göster
# π city , country ( σ population > 1000000(cities) ) 
run_query("""
            select city , country from cities
            where population > 1000000
            limit 5;
        """)

In [ ]:
#Türkiye'de nüfusu 500.000'den fazla olan en kalabalık 6 şehri al
# π city,country,population ( σ country = 'Turkey ∧ σ population > 500000 (cities) )
run_query("""
          SELECT city, country, population
          from cities
          where country = 'Turkey' AND population > 500000
          order by population DESC
          limit 6;
        """
)

In [ ]:
#almanyanın en az nüfuslu 3 şehri
# π city,population ( σ country='Germany' (cities) )
run_query("""select city , population
          from cities
          where country ='Germany'
          order by population asc
          limit 3;
          """)

In [ ]:
#ülkelerin toplam nüfusu top5
# π country,sum(population)->total_population (cities)
run_query("""
select country , sum(population) as total_population
          from cities
          group by country
          order by total_population desc
          limit 5;
""")

In [ ]:
#nüfusu 1000 ile 2000 arasında olan şehirlerden en düşük nüfuslu ilk 7 tanesini al   
# π country,city,population ( σ population>1000 ∧ population<2000 (cities) )
run_query("""select country, city, population
          from cities
          where population between 1000 and 2000 
          order by population asc
          limit 7;
          """)

In [ ]:
#a ile başlayıp a ile biten ülkeler
# π country ( σ country like 'A%' ∧ σ country like '%A' (cities) )
run_query("select country from cities where country like 'A%' and country like '%A' group by country;")

In [ ]:
#her ülkede kaç tane şehir kayıtlı
run_query("select country,count(*) as cityCount from cities group by country order by cityCount desc;")

In [ ]:
#ülkelere göre ortalama şehir nüfusları
run_query("select country, avg(population)as avgPop from cities group by country order by avgPop desc;")

In [ ]:
#her ülkenin en kalabalık şehrini bulma
run_query("select country,max(population) as max_şehir from cities group by co")

In [ ]:
#kayıtlı şehir sayısı 
# π count(id)->total_cities ​(cities)
run_query("select count(id) as total_cities from cities;")
print("\n")
#kayıtlı şehirlerin ortalama nüfusu nedir 
# π avg(population)->avarage_population (cities)
run_query("select avg(population) as avarage_population from cities;")
print("\n")
#tokyo şehrinin nüfusu
# π population->populasyon ( σ city = 'Tokyo' (cities) )
run_query("select population as populasyon from cities where city = 'Tokyo' ;")
print("\n")
#en yüksek rakımlı şehir
# π country,city,lat->latitude ( cities )
run_query("select country, city, lat as latitude from cities order by lat desc limit 1 ;")

In [ ]:
#öğrenci ders not JOIN sorgusu
run_query("""
            SELECT
                o.ad,
                o.soyad,
                d.ders_adi,
                a.notu
            FROM AkademikKayitlar a
            JOIN Ogrenciler o ON a.ogrenci_id = o.ogrenci_id
            JOIN Dersler d ON a.ders_id = d.ders_id;
""")

In [ ]:
#ders bazlı ortalama not GROUP BY sorgusu
run_query("""
            SELECT
                d.ders_adi,
                AVG(a.notu) AS ortalama_not
            FROM AkademikKayitlar a
            JOIN Dersler d ON a.ders_id = d.ders_id
            GROUP BY d.ders_adi;
""")

In [ ]:
#ortalama notu 80 üzeri olan dersler HAVING sorgusu
run_query("""
            SELECT
                d.ders_adi,
                AVG(a.notu) AS ortalama
            FROM AkademikKayitlar a
            JOIN Dersler d ON a.ders_id = d.ders_id
            GROUP BY d.ders_adi
            HAVING AVG(a.notu) > 80;
""")

bağlantıyı kapat

In [ ]:
connection.close()